In [58]:
import pandas as pd
import numpy as np
import json
#import matplotlib.pyplot as plt
#import seaborn as sns
import collections

## Load Json file

In [2]:
with open('data_analysis.json') as radius_data:
   data = json.load(radius_data)
   df = pd.DataFrame(data)
#df=pd.read_json("/Users/Yu/Downloads/data_analysis.json")
#pd.DataFrame(df)

In [3]:
df.shape

(1000000, 10)

In [4]:
df.head()

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
0,10085 SCRIPPS RANCH CT STE A,44420000,SAN DIEGO,50 to 99,AMD CUSTOM,3123628000,$20 to 50 Million,CA,10+ years,92131
1,2566 SHALLOWFORD RD NE STE 104 # 302,31490000,ATLANTA,1 to 4,Real Hope Real Estate Inc,None,"Less Than $500,000",GA,10+ years,30345
2,212 E MAIN ST,53120000,NEOSHO,1 to 4,Jimmy Sexton Photography,4046331779,"Less Than $500,000",MO,10+ years,64850
3,6032 CHEROKEE DR,54000000,CINCINNATI,1 to 4,YOU'RE ART,4174513798,"Less Than $500,000",OH,10+ years,45243
4,1315 N WOOSTER AVE,54100000,STRASBURG,1 to 4,Hayberg Restoration Network LLC,5135612584,"$500,000 to $1 Million",OH,10+ years,44680


In [5]:
df.dtypes

address             object
category_code       object
city                object
headcount           object
name                object
phone               object
revenue             object
state               object
time_in_business    object
zip                 object
dtype: object

In [6]:
# check number of unique values of each field
# None value not included
df.apply(pd.Series.nunique)

address             892120
category_code         1184
city                 13720
headcount               15
name                890723
phone               575154
revenue                 17
state                   59
time_in_business        11
zip                  26397
dtype: int64

In [17]:
# check what categories each field has
a = df.zip.unique()
a.sort()
a
#df.address.unique()
#df.category_code.unique()
#df.state.unique()
#df.city.unique()
#df.headcount.unique()
#df.name.unique()
#df.phone.unique()
#df.revenue.unique()
#df.state.unique()
#df.zip.unique()

array([None, 0, u'', ..., u'99929', u'none', u'null'], dtype=object)

## Calcualting fill rate and create summary table

In [18]:
# use filter(None,..) to detect all false values for all string objects
cols = df.columns.tolist()
list1 = []
list2 = []
for i in cols:
    feat = i
    filtered = filter(None, df[i]).__len__()
    list1.append(feat)
    list2.append(filtered)
df1 = pd.DataFrame({'features': list1, 'filled_num': list2})
del list1, list2

In [19]:
df1
#number of rows filled for each column (exclude false values: None, 0, '')

,features,filled_num
0,address,999957
1,category_code,999962
2,city,999950
3,headcount,962334
4,name,999964
5,phone,590859
6,revenue,943066
7,state,999961
8,time_in_business,916107
9,zip,999955


In [20]:
# show numbers of rows for each column excluding None values only
df2= df.count().to_frame().reset_index()
df2 = df2.rename(columns= {'index': 'features',0: 'not_None_value'})

In [21]:
df2

,features,not_None_value
0,address,999986
1,category_code,999986
2,city,999986
3,headcount,962352
4,name,999986
5,phone,590889
6,revenue,943092
7,state,999986
8,time_in_business,916125
9,zip,999988


In [22]:
df.isnull().sum()
# so isnull only detect cells filled with None value

address                 14
category_code           14
city                    14
headcount            37648
name                    14
phone               409111
revenue              56908
state                   14
time_in_business     83875
zip                     12
dtype: int64

In [23]:
df[df['category_code'].isnull()]

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
10787,194 COUNTY ROAD 627,None,PHILLIPSBURG,100 to 249,Webster University,8587557119,$50 to 100 Million,NJ,10+ years,08865
39327,152 MAGNOLIA ST,None,SPARTANBURG,1 to 4,Rowlands T-V,None,"Less Than $500,000",SC,10+ years,29306
121586,815 N POTTSTOWN PIKE,None,EXTON,1 to 4,Duncan Aviation Inc,6039340048,$1 to 2.5 Million,PA,10+ years,19341
163261,444 HIGUERA ST FL 3,None,SAN LUIS OBISPO,1 to 4,Norbac III International Co,6142216766,"Less Than $500,000",CA,10+ years,93401
463128,8119 ARDLEIGH ST,None,PHILADELPHIA,20 to 49,Chason Service Engineers Inc,None,$10 to 20 Million,PA,6-10 years,19118
654619,2380 ENBORG LN,None,SAN JOSE,1 to 4,Nac Tel,None,"Less Than $500,000",CA,10+ years,95128
734229,3899 S SUNCOAST BLVD STE 100,None,HOMOSASSA,1 to 4,Regions Bank,5617929161,"Less Than $500,000",FL,6-10 years,34448
741746,3923 WILLAMETTE AVE SE,None,ALBANY,1 to 4,Metromont Corporation,4087484400,"Less Than $500,000",OR,6-10 years,97322
794438,5413 BICENTENNIAL DR,None,MOUNT MORRIS,10 to 19,Veterinary,None,$2.5 to 5 Million,MI,6-10 years,48458
813712,6837 SHOREWAY DR,None,GRAND PRAIRIE,1 to 4,COSTELLO & GREYDANUS,None,$5 to 10 Million,TX,10+ years,75054


In [24]:
# create a dataframe for summary
df1['total_row_num'] = df.shape[0]
df3 = pd.merge(df1, df2, on='features')
df_Q1 = df3[['features', 'total_row_num','filled_num']]
df_Q1.dtypes

features         object
total_row_num     int64
filled_num        int64
dtype: object

In [25]:
# Q1: calculate fill rate (define records with [None, '', 0] as not having a value )
df_Q1['total_row_num'] = df_Q1['total_row_num'].astype(float)
df_Q1['filled_num'] = df_Q1['filled_num'].astype(float)
df_Q1['not_filled_num'] =  df_Q1['total_row_num'] - df_Q1['filled_num']
df_Q1['fill_rate'] = df_Q1['filled_num']/df_Q1['total_row_num']

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [26]:
df_Q1

,features,total_row_num,filled_num,not_filled_num,fill_rate
0,address,1000000.0,999957.0,43.0,0.999957
1,category_code,1000000.0,999962.0,38.0,0.999962
2,city,1000000.0,999950.0,50.0,0.999950
3,headcount,1000000.0,962334.0,37666.0,0.962334
4,name,1000000.0,999964.0,36.0,0.999964
5,phone,1000000.0,590859.0,409141.0,0.590859
6,revenue,1000000.0,943066.0,56934.0,0.943066
7,state,1000000.0,999961.0,39.0,0.999961
8,time_in_business,1000000.0,916107.0,83893.0,0.916107
9,zip,1000000.0,999955.0,45.0,0.999955


## Calculating true-valued fill rate

In [27]:
list_a = df.values.T.tolist()
non_valued_fills = ['null', 'none', '0', ' ','', 0, None]
for list in list_a:
    for idx, x in enumerate(list):
        if x in non_valued_fills:
            list[idx] = None
dfx= pd.DataFrame(list_a)
df4= dfx.transpose() 
my_columns = ["address", "category_code", "city", "headcount", "name", "phone", "revenue", "state", "time_in_business", "zip"]
df4.columns = my_columns
# df4 is a dataframe with only None as missing or not true-valued data, 
# all the other types of problematic data were replaced by None


In [29]:
df4[df4['address'].isnull()] #only None values

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
21773,None,54000000,CHICAGO,20 to 49,RCD Sales Co,None,"Less Than $500,000",IL,10+ years,60601
22062,None,31400000,FORT LAUDERDALE,10 to 19,"SHERMAN C. SMITH, Attorney at Law",6153839500,$20 to 50 Million,FL,10+ years,33309
33870,None,42000000,KANSAS CITY,1 to 4,NICHOLS MANAGEMENT INC,None,"Less Than $500,000",MO,10+ years,64133
43516,None,61111000,FAIRBANKS,10 to 19,University Gallery,5038884336,$2.5 to 5 Million,AK,10+ years,99701
61277,None,23820000,MOUNT LAUREL,None,Animal Bikes,None,$20 to 50 Million,NJ,6-10 years,8054
95603,None,44400000,ORANGE,5 to 9,Journeys Of The Mind Inc,6022698988,$2.5 to 5 Million,CA,6-10 years,92866
96310,None,44000000,ROCKFORD,5 to 9,Habitat For Humanity,7816391919,"$500,000 to $1 Million",IL,10+ years,61109
102461,None,44000000,CHICAGO,1 to 4,Prinz Woodworking Inc,8654064420,"$500,000 to $1 Million",IL,10+ years,60642
108363,None,62130000,GALT,1 to 4,"East Coast Roofing, Siding & Windows",None,"Less Than $500,000",CA,10+ years,95632
114195,None,53111000,WATERLOO,20 to 49,Fluid Resource Management,4084819020,$1 to 2.5 Million,IA,6-10 years,50703


In [30]:
null_count = pd.DataFrame(df4.isnull().sum())
null_count['features'] = null_count.index
null_count = null_count.reset_index(drop=True)
null_count.columns = ['not_true_sum','features']

In [31]:
# merge the true-valued data with the summary filled data 
df_Q2 = pd.merge(df_Q1, null_count, on='features')
df_Q2['not_true_sum'] = df_Q2['not_true_sum'].astype(float)
df_Q2['true_valued_sum'] = df_Q2['total_row_num'] - df_Q2['not_true_sum']
df_Q2['true_valued_fill_rate'] = df_Q2['true_valued_sum']/ df_Q2['total_row_num']
df_Q2

,features,total_row_num,filled_num,not_filled_num,fill_rate,not_true_sum,true_valued_sum,true_valued_fill_rate
0,address,1000000.0,999957.0,43.0,0.999957,102.0,999898.0,0.999898
1,category_code,1000000.0,999962.0,38.0,0.999962,90.0,999910.0,0.999910
2,city,1000000.0,999950.0,50.0,0.999950,105.0,999895.0,0.999895
3,headcount,1000000.0,962334.0,37666.0,0.962334,37727.0,962273.0,0.962273
4,name,1000000.0,999964.0,36.0,0.999964,90.0,999910.0,0.999910
5,phone,1000000.0,590859.0,409141.0,0.590859,409202.0,590798.0,0.590798
6,revenue,1000000.0,943066.0,56934.0,0.943066,56999.0,943001.0,0.943001
7,state,1000000.0,999961.0,39.0,0.999961,104.0,999896.0,0.999896
8,time_in_business,1000000.0,916107.0,83893.0,0.916107,83952.0,916048.0,0.916048
9,zip,1000000.0,999955.0,45.0,0.999955,110.0,999890.0,0.999890


## Calculating cardinality

In [32]:
unique = pd.DataFrame(df4.apply(pd.Series.nunique))
unique.columns = ['Cardinality']

In [33]:
unique['total_row_num'] = df.shape[0]
unique

,Cardinality,total_row_num
address,892114,1000000
category_code,1178,1000000
city,13714,1000000
headcount,9,1000000
name,890717,1000000
phone,575148,1000000
revenue,11,1000000
state,53,1000000
time_in_business,5,1000000
zip,26391,1000000


## Exploring data for more info

### Analysis of location related fields (address, city, state, zip)

In [34]:
df_Q2
# noticed that these 4 fields have similar amount of missing values (102, 105, 104, 110)
# questions: are these missing from almost the same businesses?
# which means we lost the location info of the businesses?

,features,total_row_num,filled_num,not_filled_num,fill_rate,not_true_sum,true_valued_sum,true_valued_fill_rate
0,address,1000000.0,999957.0,43.0,0.999957,102.0,999898.0,0.999898
1,category_code,1000000.0,999962.0,38.0,0.999962,90.0,999910.0,0.999910
2,city,1000000.0,999950.0,50.0,0.999950,105.0,999895.0,0.999895
3,headcount,1000000.0,962334.0,37666.0,0.962334,37727.0,962273.0,0.962273
4,name,1000000.0,999964.0,36.0,0.999964,90.0,999910.0,0.999910
5,phone,1000000.0,590859.0,409141.0,0.590859,409202.0,590798.0,0.590798
6,revenue,1000000.0,943066.0,56934.0,0.943066,56999.0,943001.0,0.943001
7,state,1000000.0,999961.0,39.0,0.999961,104.0,999896.0,0.999896
8,time_in_business,1000000.0,916107.0,83893.0,0.916107,83952.0,916048.0,0.916048
9,zip,1000000.0,999955.0,45.0,0.999955,110.0,999890.0,0.999890


In [35]:
df4.isnull().sum()

address                102
category_code           90
city                   105
headcount            37727
name                    90
phone               409202
revenue              56999
state                  104
time_in_business     83952
zip                    110
dtype: int64

In [36]:
# get index of None values of the 4 fields to see if they came from the same obs
index1 = df4['address'].index[df4['address'].isnull()]
index1

Int64Index([ 21773,  22062,  33870,  43516,  61277,  95603,  96310, 102461,
            108363, 114195,
            ...
            888450, 902912, 912811, 912945, 919817, 920461, 938708, 950741,
            958904, 964271],
           dtype='int64', length=102)

In [37]:
index2 = df4['city'].index[df4['city'].isnull()]
index2

Int64Index([  3060,  16781,  28060,  44929,  47853,  50521,  52597,  56785,
             76271,  85507,
            ...
            861453, 863034, 872059, 876312, 903644, 929904, 942806, 964694,
            983425, 991943],
           dtype='int64', length=105)

In [38]:
index3 = df4['state'].index[df4['state'].isnull()]
index4 = df4['zip'].index[df4['zip'].isnull()]

In [40]:
index3

Int64Index([  8036,  10218,  25897,  35014,  47706,  52935,  57841,  61757,
             66211,  69402,
            ...
            852829, 875274, 880800, 893167, 899098, 911994, 916963, 938551,
            959664, 990946],
           dtype='int64', length=104)

In [44]:
# index back to the data recordings
#df.index[df['address'] == 0].tolist()
#index= df.index[df['address'] == ' '].tolist()
#df4.loc[index1] #address 102x10
#df4.loc[index2] #city 105x10
#df4.loc[index3] #state 104x10
df4.loc[index4] #zip 110x10

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
9831,2111 NEW HOPE CHURCH RD,33351000,RALEIGH,None,Phoenix Central School System,8882688267,$5 to 10 Million,NC,10+ years,None
19529,19555 GLENVIEW CT,54111000,OREGON CITY,50 to 99,Wilklow Orchards,None,"Less Than $500,000",OR,10+ years,None
35388,8101 S 15TH ST,52411000,LINCOLN,50 to 99,Protech Networks Inc,8606880888,$5 to 10 Million,NE,None,None
47917,1550 17TH ST STE 500,81121000,DENVER,20 to 49,Totaro Totaro and Siegel Inc.,9894275046,"$500,000 to $1 Million",CO,10+ years,None
54451,250 E WISCONSIN AVE STE 1800,23611600,MILWAUKEE,1 to 4,Gammons Gulch,3034691781,$5 to 10 Million,WI,10+ years,None
69058,905 N MARKET ST,42000000,SHREVEPORT,10 to 19,Sauer Land Surveying,3028923540,$10 to 20 Million,LA,10+ years,None
73707,338 THOMPSON RD,53119000,WEBSTER,5 to 9,Lincoln RD Chiropractic,3015930506,"Less Than $500,000",MA,6-10 years,None
87572,19603 S VERMONT AVE,23610000,TORRANCE,10 to 19,U.S. Lawns - Gloucester,None,$5 to 10 Million,CA,None,None
98992,2975 TREAT BLVD STE B5,54180000,CONCORD,1 to 4,HELP YOU WELL,None,"Less Than $500,000",CA,10+ years,None
116097,6200 S TRANSIT RD,22000000,LOCKPORT,1 to 4,In Harmony Pediatric Therapy,None,$20 to 50 Million,NY,10+ years,None


### Analysis of headcount and time_in_business relationship

In [45]:
df_Q4_1 = df4[['time_in_business','headcount','name']].groupby(['time_in_business', 'headcount']).count()

In [46]:
# to see the if the headcount will increase as time in business increase
df_Q4_1

name
time_in_business headcount         
1-2 years        1 to 4        4494
                 10 to 19      1945
                 100 to 249     476
                 20 to 49      1568
                 250 to 499     146
                 5 to 9        2730
                 50 to 99       757
                 500 to 999      68
                 Over 1,000      84
10+ years        1 to 4      271643
                 10 to 19    114928
                 100 to 249   27713
                 20 to 49     92080
                 250 to 499    8554
                 5 to 9      161000
                 50 to 99     46100
                 500 to 999    4015
                 Over 1,000    4179
3-5 years        1 to 4       13889
                 10 to 19      5761
                 100 to 249    1342
                 20 to 49      4581
                 250 to 499     392
                 5 to 9        8179
                 50 to 99      2249
                 500 to 999     193
                 Over 1,000     223
6-10 years       1 to 4       38249
                 10 to 19     15959
                 100 to 249    3893
                 20 to 49     12765
                 250 to 499    1155
                 5 to 9       22581
                 50 to 99      6440
                 500 to 999     552
                 Over 1,000     618

In [47]:
df_Q4_2 = df4['name']
dups_name = [item for item, count in collections.Counter(df_Q4_2).items() if count > 1]
print dups_name

[u'Ginop Sales Inc', u'L-3 COMMUNICATIONS', u'TLT-BABCOCK INC', u'EMERSON PROCESS MANAGEMENT', u'Maine Country & Coast Real Estate', u'Barnhill Contracting Co', u'Salient Inc', u'Resort Realty', u'Dorris Bruce E Appraisal', u'Hampton Boat & RV Storage', u'United Computer Warehouse', u'Pilot Freight Services', u'FREETOWN-LAKEVILLE REGIONAL SCHOOL DISTRICT', u'West Bloomfield School District', u"TAMURA'S FINE WINE & LIQUORS", u'Watson Electrical Construction Co LLC', u'Sentara Healthcare', u'Edge Group', u'RE/MAX AROUND ATLANTA EAST', u'Office Resources Inc', u'Centerpoint', u'Alpine Home Medical', u'SIMS GROUP', u'Sutherland Asbill & Brennan LLP', u"JIM'S BARBER SHOP", u'Beacon Financial Advisors', u'Caribou Coffee', u'S3 Pump Service', u'Grifols', u'DONOHUE & ASSOC.', u'Trinity Title of Texas LLC', u'Metal Products Co', u'Hopes Inc', u'Coastal Commerce Bank', u'Coram Specialty Infusion Services', u'Chelsea Technology Inc', u'Half Hollow Hills Central School District', u'MTRMLS, INC. D/

In [55]:
df4[df4['name'] == 'ARIZONA CHEMICAL COMPANY LLC']

,address,category_code,city,headcount,name,phone,revenue,state,time_in_business,zip
223973,24901 NORTHWESTERN HWY,51000000,SOUTHFIELD,5 to 9,ARIZONA CHEMICAL COMPANY LLC,3125041323,$1 to 2.5 Million,MI,10+ years,48075
544101,17 PARK ST,54111000,WANAQUE,10 to 19,ARIZONA CHEMICAL COMPANY LLC,None,"Less Than $500,000",NJ,10+ years,07465


In [56]:
unique.to_csv('unique.csv')

In [57]:
df_Q4_1.to_csv('df_Q4_1.csv')

In [64]:
df4.to_csv('data.csv', header=True, index=False, encoding='utf-8')